In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double
from libc.math cimport sqrt, pow, abs


cdef void mean_sd_1d(const double[:] x, double* mean, double* std):
    cdef Py_ssize_t n = x.size, i

    cdef double k = x[0]
    cdef double Ex = 0., Ex2 = 0.
    mean[0] = 0.

    for i in range(n):
        mean[0] += x[i]
        Ex += x[i] - k
        Ex2 += (x[i] - k)**2
    
    std[0] = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mean[0] /= n


def RatioBeyondRSigma(const double[:, :, :] x, double r):
    cdef Py_ssize_t m = x.shape[0], n = x.shape[1], p = x.shape[2], i, j, k
    xcount = zeros((m, p), dtype=npy_double, order='C')
    cdef double[:, ::1] count = xcount
    cdef double mu = 0., sigma = 0.

    for i in range(m):
        for k in range(p):
            mu = 0.
            sigma = 0.
            mean_sd_1d(x[i, :, k], &mu, &sigma)
            sigma *= r
            for j in range(n):
                if (abs(x[i, j, k] - mu) > sigma):
                    count[i, k] += 1
    for i in range(m):
        for j in range(p):
            count[i, j] /= n
    return xcount

In [4]:
from ratiobeyond import ratiobeyondrsigma as frbrs

In [5]:
x = np.random.rand(50000, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

t = np.arange(0, 150/50, 1/50)

In [6]:
%timeit RatioBeyondRSigma(x, 0.5)

130 ms ± 31.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit frbrs(xf, 0.5)

92.4 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
np.allclose(frbrs2(xf, 0.5).T, RatioBeyondRSigma(x, 0.5))

True